# Тестирование

## T5

In [1]:
import pandas as pd
from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch
from tqdm.auto import tqdm, trange
import gc
import csv

In [4]:
model = T5ForConditionalGeneration.from_pretrained(r'C:\Users\user\Desktop\try1\t5_small_gsd_kmeans__results\t5_small_cl_train_38000')

base_model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [5]:
def generate(text, model, n=None, max_length='auto', temperature=0.0, beams=3):
    texts = [text] if isinstance(text, str) else text
    
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(model.device)
    
    if max_length == 'auto':
        max_length = int(inputs.shape[1] * 1.2) + 10
    
    result = model.generate(
        inputs, 
        num_return_sequences=n or 1, 
        do_sample=False, 
        temperature=temperature, 
        max_length=max_length,
        repetition_penalty=3.0,
        num_beams=beams,
    )
    
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    
    if not n and isinstance(text, str):
        return texts[0]
    return texts

In [7]:
print(generate(['оставаться'], model, temperature=50.0, beams=10))

['оратс,VERB,Imp,Ind,S']


In [8]:
print(generate(['кошка'], model, temperature=50.0, beams=10))

['кока,NOUN,Inan,Nom,F']


In [9]:
path_to_test = r'C:\Users\user\Desktop\try1\testing\gsd_test_data.csv'

In [10]:
old_df = pd.read_csv(path_to_test)
old_df

,form,data
0,Билли,"Билли,PROPN,Anim,Nom,Masc,Sing"
1,начал,"начать,VERB,Perf,Masc,Ind,Sing,Past,Fin,Act"
2,играть,"играть,VERB,Imp,Inf,Act"
3,за,"за,ADP,None"
4,резервный,"резервный,ADJ,Inan,Acc,Pos,Masc,Sing"
...,...,...
11380,Землю,"земля,NOUN,Inan,Acc,Fem,Sing"
11381,ради,"ради,ADP,None"
11382,получения,"получение,NOUN,Inan,Gen,Neut,Sing"
11383,энергии,"энергия,NOUN,Inan,Gen,Fem,Sing"


In [11]:
forms = old_df['form'].to_list()
forms

['Билли',
 'начал',
 'играть',
 'за',
 'резервный',
 'состав',
 '``',
 'Черка',
 '&#39;&#39;',
 'в',
 'возрасте',
 '16',
 'лет',
 ',',
 'а',
 'через',
 'пару',
 'сезонов',
 'был',
 'приглашён',
 'в',
 'основной',
 'состав',
 '.',
 'Стоимость',
 'проезда',
 'с',
 '5',
 'января',
 '2013',
 'года',
 '--',
 '15',
 'рублей',
 ',',
 'движение',
 'осуществляется',
 'с',
 '6.00',
 'до',
 '00.20',
 '.',
 'Стал',
 'членом',
 'секретариата',
 'общественной',
 'безопасности',
 'Мексики',
 '(',
 'SSP',
 ',',
 'Secretaría',
 'de',
 'Seguridad',
 'Pública',
 ')',
 'и',
 'специальным',
 'уполномоченным',
 'Федеральной',
 'полиции',
 'Мексики',
 '(',
 'PFP',
 ',',
 'Policía',
 'Federal',
 'Preventiva',
 ')',
 '.',
 'Официальный',
 'код',
 'округа',
 'AT125',
 '.',
 'Здесь',
 'обитает',
 'несколько',
 'десятков',
 'видов',
 'птиц',
 '.',
 'Когда',
 'Мэлоуну',
 'исполнилось',
 'три',
 'года',
 ',',
 'его',
 'отец',
 ',',
 'Шедрик',
 'Хэй',
 ',',
 'ушел',
 'из',
 'семьи',
 'и',
 'женился',
 'на',
 'другой

In [12]:
%%time
predictions = []
for form in forms:
    pred = generate(form, model, temperature=50.0, beams=10)
    predictions.append(pred)

CPU times: total: 8h 18min 39s
Wall time: 1h 2min 42s


In [13]:
old_df['predicted_data'] = predictions
old_df

,form,data,predicted_data
0,Билли,"Билли,PROPN,Anim,Nom,Masc,Sing","илли,PROPN,Anim,Nom,Ma"
1,начал,"начать,VERB,Perf,Masc,Ind,Sing,Past,Fin,Act","наал,NOUN,Inan,Nom,M"
2,играть,"играть,VERB,Imp,Inf,Act","ират,VERB,Imp,Ind,S"
3,за,"за,ADP,None","а,ADP,None"
4,резервный,"резервный,ADJ,Inan,Acc,Pos,Masc,Sing","авлт,VERB,Imp,Ind,"
...,...,...,...
11380,Землю,"земля,NOUN,Inan,Acc,Fem,Sing","емл, велиитемн,"
11381,ради,"ради,ADP,None","ради,NOUN,Inan,Loc"
11382,получения,"получение,NOUN,Inan,Gen,Neut,Sing","олуени,лиие, восоа"
11383,энергии,"энергия,NOUN,Inan,Gen,Fem,Sing","тот,DET,Nom,Masc,Sing"


In [14]:
old_df.to_csv(r'C:\Users\user\Desktop\try1\testing\gsd_test_kmeans.csv')

## RuPrompts

In [15]:
from transformers import GPT2LMHeadModel, AutoTokenizer

In [16]:
backbone_id = "sberbank-ai/rugpt3large_based_on_gpt2"

model = GPT2LMHeadModel.from_pretrained(backbone_id)
tokenizer = AutoTokenizer.from_pretrained(backbone_id, pad_token="<pad>", eos_token="<pad>")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
from transformers import pipeline
from ruprompts import Prompt

prompt = Prompt.from_pretrained(r"C:\Users\user\Desktop\try1\checkpoint-80850")

ppln = pipeline("text2text-generation-with-prompt", prompt=prompt, model=model, tokenizer=tokenizer)

In [18]:
ppln({"form": "Ублюдок"}, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'NOUN,Anim,Nom,Masc,Sing'}]

In [19]:
path_to_test = r'C:\Users\user\Desktop\try1\testing\gsd_test_data.csv'

In [21]:
old_df2 = pd.read_csv(path_to_test)
forms2 = old_df2['form'].to_list()
old_df2

,form,data
0,Билли,"Билли,PROPN,Anim,Nom,Masc,Sing"
1,начал,"начать,VERB,Perf,Masc,Ind,Sing,Past,Fin,Act"
2,играть,"играть,VERB,Imp,Inf,Act"
3,за,"за,ADP,None"
4,резервный,"резервный,ADJ,Inan,Acc,Pos,Masc,Sing"
...,...,...
11380,Землю,"земля,NOUN,Inan,Acc,Fem,Sing"
11381,ради,"ради,ADP,None"
11382,получения,"получение,NOUN,Inan,Gen,Neut,Sing"
11383,энергии,"энергия,NOUN,Inan,Gen,Fem,Sing"


In [ ]:
%%time
predictions2 = []
for form in forms2:
    pred = ppln({"form": form}, do_sample=True)
    print(pred)
    predictions2.append(pred)

In [ ]:
old_df2['predicted_data'] = predictions2

In [ ]:
old_df2.to_csv(r'C:\Users\user\Desktop\try1\testing\gsd_test_rupr_kmeans.csv')